# Importing data

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1GhyH4k9C4uPRnMAMKhJYOqa-V9Tqt4q8'
train1 = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))

# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1q0jIkWkePF509qtHzynMBjrDUIgNV2uk'
test1 = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))


# Importing Libraries

In [0]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
!pip install torch
!pip install flair
import torch
import seaborn as sns
import flair
import nltk
import re
import torch
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

    100% |████████████████████████████████| 591.8MB 32kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x6256e000 @  0x7fd0a50d32a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 71kB 6.1MB/s 
    100% |████████████████████████████████| 22.6MB 2.2MB/s 
    100% |████████████████████████████████| 51kB 26.5MB/s 
    100% |████████████████████████████████| 12.8MB 4.0MB/s 
    100% |████████████████████████████████| 798kB 16.1MB/s 
  Running setup.py bdist_wheel for flair ... - \ done
  Stored in directory: /root/.cache/pip/wheels/54/1c/24/39ec59521bb1a83306cf5224a50f99aad60b55dd3b42d32ed0
  Running setup.py bdist_wheel for segtok ... - done
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f734

In [0]:
### Importing the Dataset
import io
data = pd.read_csv(io.StringIO(train1.GetContentString()))  ## contain test+train tweets
print(data.head())


2019-01-21 11:27:51,155 file_cache is unavailable when using oauth2client >= 4.0.0
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named '

In [0]:
text = data['tweet'] #extracting the text part
text.head()

0      user when a father is dysfunctional and is s...
1     user  user thanks for  lyft credit i can t us...
2                                  bihday your majesty
3     model   i love u take with u all the time in ...
4               factsguide  society now     motivation
Name: tweet, dtype: object

In [0]:
k = text.tolist() ## Text to python list for iteration
print(k[:10])

['  user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction     run', ' user  user thanks for  lyft credit i can t use cause they don t offer wheelchair vans in pdx      disapointed  getthanked', '  bihday your majesty', ' model   i love u take with u all the time in ur                                      ', ' factsguide  society now     motivation', '      huge fan fare and big talking before they leave  chaos and pay disputes when they get there   allshowandnogo  ', '  user camping tomorrow  user  user  user  user  user  user  user danny   ', 'the next school year is the year for exams      can t think about that       school  exams    hate  imagine  actorslife  revolutionschool  girl', 'we won    love the land     allin  cavs  champions  cleveland  clevelandcavaliers      ', '  user  user welcome here    i m   it s so  gr    ']


# Embeddings

In [0]:
from flair.embeddings import WordEmbeddings
from flair.embeddings import CharacterEmbeddings
from flair.embeddings import StackedEmbeddings
from flair.embeddings import FlairEmbeddings

# init standard GloVe embedding
#glove_embedding = WordEmbeddings('glove')
# init standard character embeddings
#character_embeddings = CharacterEmbeddings()
#twitter = WordEmbeddings('twitter')
flair_forward  = FlairEmbeddings('news-forward-fast')
flair_backward = FlairEmbeddings('news-backward-fast')

stack = StackedEmbeddings( embeddings = [ flair_forward, flair_backward ])


2019-01-21 11:27:52,443 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-forward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpgldatrmt


100%|██████████| 19689779/19689779 [00:00<00:00, 47207378.74B/s]

2019-01-21 11:27:53,011 copying /tmp/tmpgldatrmt to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt
2019-01-21 11:27:53,038 removing temp file /tmp/tmpgldatrmt


2019-01-21 11:27:56,003 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpwc80y5up


100%|██████████| 19689779/19689779 [00:00<00:00, 40716062.47B/s]

2019-01-21 11:27:56,760 copying /tmp/tmpwc80y5up to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt


2019-01-21 11:27:56,795 removing temp file /tmp/tmpwc80y5up


## Flair types

In [0]:
from flair.data import Sentence

# create a sentence
sentence = Sentence('The grass is green .')

# embed words in sentence
stack.embed(sentence)
for token in sentence:
  print(token.embedding)

tensor([ 3.0558e-03, -8.9402e-08,  3.2061e-07,  ..., -1.6456e-09,
        -7.8441e-05,  1.6318e-02])
tensor([ 8.3728e-03, -8.8324e-07,  6.8124e-07,  ..., -1.9470e-06,
         6.3773e-04,  5.7139e-03])
tensor([ 2.2655e-03, -5.2416e-06,  5.9481e-07,  ..., -9.3616e-07,
         1.6768e-05,  2.9047e-04])
tensor([ 7.8900e-04, -8.2879e-07,  1.1457e-06,  ..., -4.5412e-08,
        -1.1528e-04,  3.4503e-02])
tensor([ 1.7806e-03,  9.4982e-08,  7.0174e-08,  ..., -1.0784e-08,
        -4.4279e-06,  2.0673e-03])


In [0]:
# Size of embedding
z = token.embedding.size()[0]
print(z)

2048


In [0]:
# Calculating Document Embeddin

s = torch.zeros(0,z)    # creating a tensor for storing sentence embeddings

for tweet in tqdm(k):   # iterating Sentences
  w = torch.zeros(0,z)   # tensor for words
  sentence = Sentence(tweet)
  stack.embed(sentence)
  for token in sentence:
    w = torch.cat((w,token.embedding.view(-1,z)),0)   # storing word Embeddings in a sentence
  s = torch.cat((s, w.mean(dim = 0).view(-1, z)),0)   # storing sentence Embeddings

    

 90%|█████████ | 44479/49159 [53:51<08:58,  8.69it/s]

In [0]:
s.size()   # size of Document Embedding

# Partitioning of data

In [0]:
X = s.numpy()                 # Bulk Data
test = X[31962:,:]
train = X[:31962,:]
target = data['label'][data['label'].isnull()==False].values


# Implementing XGBoost

In [0]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score



x_train, x_valid, y_train, y_valid = train_test_split(train, target,  
                                                          random_state=42, 
                                                          test_size=0.3)


dtrain = xgb.DMatrix(x_train,y_train)          # XGB compatible data
dvalid = xgb.DMatrix(x_valid, label = y_valid)

### Custom Evaluator

In [0]:
def custom_eval(preds, dtrain):
    labels = dtrain.get_label().astype(np.int)
    preds = (preds >= 0.3).astype(np.int)
    return [('f1_score', f1_score(labels, preds))]

### parameters

In [0]:
params = {
    'colsample': 0.9,
 'colsample_bytree': 0.5,
 'eta': 0.1,
 'max_depth': 8,
 'min_child_weight': 6,
 'objective': 'binary:logistic',
 'subsample': 0.9
}

In [0]:
xgb_model = xgb.train(
    params,
    dtrain,
    feval= custom_eval,
    num_boost_round= 1000,
    maximize=True,
    evals=[(dvalid, "Validation")],
    early_stopping_rounds=30
)


In [0]:
# Reformating test set fo XGB
dtest = xgb.DMatrix(test)

In [0]:
predict = xgb_model.predict(dtest) # predicting

In [0]:
predict.shape, test.shape

In [0]:
k = range(31963,49160) ## id for submission file

predictions are in form of Probability :  
Converting them to Labels using various threshold.

In [0]:
th = [0.1,0.2,0.3,0.4]
for i in th:
  extract1 = []               # Stores labels generated
  for j in predict:
    if j<i:                    
      extract1.append(0)
    else:
      extract1.append(1)
  final = pd.DataFrame({ 'id'   : k,
                         'label': extract1,})
  final = final.set_index('id')
  from google.colab import files
  final.to_csv('stack'+str(i)+'.csv')
  files.download('stack'+str(i)+'.csv')